<a href="https://colab.research.google.com/github/dkswndms4782/Deep-Learning/blob/main/knn_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
drive.mount('/content/gdrive')

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from decimal import Decimal
import warnings
warnings.filterwarnings('ignore')

column = pd.read_csv('./gdrive/My Drive/HW2/experiments.csv',header = None)
ribo_data = pd.read_csv('./gdrive/My Drive/HW2/ribo-data.csv',header = None)
nonribo_data = pd.read_csv('./gdrive/My Drive/HW2/nonribo-data.csv',header = None)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def make_arr(data,isribo):
   sz = len(data)
   data1 = [isribo for i in range(sz)]
   data1_ = [i for i in range(sz)]#index 0번째 부터 매기기 위해 0부터 sz까지의 데이터 만듦
   temp = pd.Series(data1)
   temp_ = pd.Series(data1_)
   data["index"] = temp_# index만들기
   data['isRibo'] = temp

In [ ]:
 make_arr(ribo_data,1)
 make_arr(nonribo_data,0)

In [ ]:
def knn(ribo_data,nonribo_data,test_data,k,p):
 ribo = ribo_data.to_numpy(dtype = Decimal)   # numpy배열로 바꾸기. 데이터 타임 Decimal로 안해주면 e지수표기법으로 나타나서 해줌.
 nonribo = nonribo_data.to_numpy(dtype = Decimal)
 test = test_data.to_numpy(dtype = Decimal)
 train = np.concatenate((ribo,nonribo),axis = 0) # ribo 와 nonribo합치기.

 x_train = train[0:,0:80] #X_train에는 데이터의 특성을 담음
 y_train = train[0:,80:81] #y_train에는 리보솜인지 아닌지. 
 x_test = test[0: , 0:80]
 y_test = test[0: , 80:81]

 # """테스트 세트 X_test의 각 점들마다,
  #      1) 학습 세트에 있는 모든 점들과의 거리를 계산.
   #     2) 계산된 거리들 중에서 가장 짧은 거리 k개를 선택.
    #    3) k개 선택된 레이블들 중에서 리보솜인 것들이 p 퍼센트를 넘는지 안넘는지로 리보솜인지 아닌지의 값을 판단 함."""
       
 predicts = []
 for test_pt in test:
    temp = []
    for train_pt in train:
      d = np.sqrt(np.sum(train_pt - test_pt)**2) #유클리드 거리측정법으로 거리 측정
      temp.append((d,train_pt[79],train_pt[80])) # index와 리보솜인지 아닌지의 값을 temp배열에 넣어줌
    temp.sort()
    tmp = temp[:k] #k개까지만 추출
    count_ribo = 0
    count_nonribo = 0
    for tmp_pt in tmp:
      if tmp_pt[2] == 0:
        count_nonribo += 1
      elif tmp_pt[2] == 1:
        count_ribo += 1
    if round(count_ribo / (count_ribo + count_nonribo),2) >= p: #리봄솜의 퍼센트가 p를 넘으면 1로 할당
      predicts.append((test_pt[79],1))
    elif round(count_ribo / (count_ribo + count_nonribo),2) < p:#아니면 0으로 할당
      predicts.append((test_pt[79],0))

 count_corect_ribo = 0
 count_corect_nonribo = 0
 count_just_ribo = 0
 count_just_nonribo = 0
 count = 0
 n = []

 for pred in predicts: #TN , TP, FP , FN개수 세기
  if pred[1] == 1:
    if test[count][80] == 1:
      count_corect_ribo += 1
    else : n.append(pred)
    count_just_ribo += 1
  elif pred[1] == 0:
    if test[count][80] == 0:
      count_corect_nonribo += 1
    count_just_nonribo += 1
    count+=1

 print(n)

 print("k: " , k , "\np: " , p , "\nsensitivity: " , #sensitivity,specitivity,accuracy print하기
       round(count_corect_ribo/count_just_ribo,2) , "\nspecificity: " ,
       round(count_corect_nonribo/count_just_nonribo,2) , "\naccuracy: ",
       round((count_corect_ribo + count_corect_nonribo)/(count_just_ribo + count_just_nonribo),2))

 return k,p, round(count_corect_ribo/count_just_ribo,2) , round(count_corect_nonribo/count_just_nonribo,2), round((count_corect_ribo + count_corect_nonribo)/(count_just_ribo + count_just_nonribo),2)

In [ ]:
#Cross-Validation 
ribo_set = []
nonribo_set = []
for i in range(0,6): #6개로 나눠서 ribo_set 과 nonribo_set으로 나눔
  ribo_set.append(ribo_data.sample(frac = round(1/6,2))) 
  nonribo_set.append(nonribo_data.sample(frac = round(1/6,2)))

knn_out = []
for i in range(0,6): 
  ribo_tmp = ribo_data
  nonribo_tmp = nonribo_data
  ribo_tmp = ribo_tmp.drop(ribo_set[i]["index"]) #6개중 1개 드롭하는 과정
  nonribo_tmp = nonribo_tmp.drop(nonribo_set[i]["index"]) 
  test_data = pd.concat([ribo_set[i],nonribo_set[i]]) #드롭한 set 하나로 합치기
  knn_out.append(knn(ribo_tmp,nonribo_tmp,test_data,5,0.75))

[(7.0, 1)]
k:  5 
p:  0.75 
sensitivity:  0.92 
specificity:  0.95 
accuracy:  0.95
[(21.0, 1), (11.0, 1), (3.0, 1), (5.0, 1), (13.0, 1), (87.0, 1), (93.0, 1)]
k:  5 
p:  0.75 
sensitivity:  0.59 
specificity:  0.95 
accuracy:  0.93
[(47.0, 1), (84.0, 1)]
k:  5 
p:  0.75 
sensitivity:  0.82 
specificity:  0.95 
accuracy:  0.95
[(84.0, 1), (2.0, 1), (92.0, 1), (90.0, 1)]
k:  5 
p:  0.75 
sensitivity:  0.71 
specificity:  0.95 
accuracy:  0.94
[(5.0, 1), (1.0, 1), (16.0, 1), (2.0, 1)]
k:  5 
p:  0.75 
sensitivity:  0.6 
specificity:  0.95 
accuracy:  0.94
[(55.0, 1), (20.0, 1), (3.0, 1), (21.0, 1), (87.0, 1)]
k:  5 
p:  0.75 
sensitivity:  0.67 
specificity:  0.95 
accuracy:  0.94


In [ ]:
  knn_out #결과

[(5, 0.75, 0.7, 0.95, 0.94),
 (5, 0.75, 0.82, 0.95, 0.95),
 (5, 0.75, 0.64, 0.95, 0.94),
 (5, 0.75, 0.89, 0.95, 0.95),
 (5, 0.75, 0.75, 0.95, 0.94),
 (5, 0.75, 0.73, 0.95, 0.94)]

In [ ]:
sensitivity_tmp = 0
specificity_tmp = 0
accuracy_tmp = 0
for i in range(0,6):
  sensitivity_tmp += knn_out[i][2]
  specificity_tmp += knn_out[i][3]
  accuracy_tmp += knn_out[i][4]

sensitivity_tmp /= 6 #sensitivity평균 구하기
specificity_tmp /= 6 #specitivity 평균
accuracy_tmp /= 6 #accuracy평균 구하기

knn_out.append((5,0.5,sensitivity_tmp,specificity_tmp,accuracy_tmp)) #knn_out의 마지막에 넣어줌